In [3]:
!pip install lazypredict

/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)


In [9]:
import lazypredict
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split, KFold
import numpy as np

## FASTEXT

In [10]:
data = pd.read_csv("datasets/edited/fasttext_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [11]:
df = pd.read_csv("datasets/edited/davidson_with_classes.csv")
del df['positive']
del df['negative']
del df['neutral']
df


,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [15]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [16]:
y = dataframe['target']
X = dataframe
del X['target']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:36<00:00,  3.34s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.30
DecisionTreeClassifier,0.99,0.88,None,0.99,1.98
LinearSVC,0.99,0.88,None,0.99,2.33
LogisticRegression,0.99,0.86,None,0.99,1.08
BaggingClassifier,0.99,0.85,None,0.99,11.97
Perceptron,0.98,0.80,None,0.98,0.47
CalibratedClassifierCV,0.99,0.80,None,0.99,10.34
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.46
NearestCentroid,0.78,0.79,None,0.80,0.17


In [19]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

Processing Fold 1 ...


100%|██████████| 29/29 [03:01<00:00,  6.25s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [02:58<00:00,  6.15s/it]


Processing Fold 3 ...


...

Processing Fold 9 ...


100%|██████████| 29/29 [02:56<00:00,  6.07s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [02:56<00:00,  6.08s/it]


,Fold,Model,Accuracy,F1-Score
0,1,LinearDiscriminantAnalysis,0.98,0.98
1,1,LinearSVC,0.99,0.99
2,1,BaggingClassifier,0.99,0.99
3,1,DecisionTreeClassifier,0.99,0.99
4,1,LogisticRegression,0.99,0.99
...,...,...,...,...
295,NaN,NaN,NaN,NaN
296,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN
298,NaN,NaN,NaN,NaN


In [20]:
result = result.dropna()
result

,Fold,Model,Accuracy,F1-Score
0,1,LinearDiscriminantAnalysis,0.98,0.98
1,1,LinearSVC,0.99,0.99
2,1,BaggingClassifier,0.99,0.99
3,1,DecisionTreeClassifier,0.99,0.99
4,1,LogisticRegression,0.99,0.99
...,...,...,...,...
245,10,AdaBoostClassifier,0.65,0.52
246,10,LGBMClassifier,0.61,0.48
247,10,LabelSpreading,0.27,0.17
248,10,LabelPropagation,0.27,0.17


In [22]:
result.to_csv("results/10-Fold-fasttext-davidson.csv")

## GLOVE

In [23]:
data = pd.read_csv("datasets/edited/glove_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,-0.11,0.31,0.35,-0.49,-0.07,0.55,-0.28,0.41,-0.01,0.26,...,-0.04,-0.28,0.10,-0.17,-0.13,0.49,-0.17,0.07,0.41,0.52
1,0.09,-0.02,0.22,-0.41,-0.36,0.09,-0.06,0.10,0.06,-0.08,...,0.25,0.12,-0.22,-0.19,-0.05,0.19,-0.33,0.03,0.43,0.54
2,0.04,0.08,0.33,-0.47,-0.28,0.48,-0.20,-0.01,0.03,-0.19,...,0.19,0.13,-0.27,-0.11,0.01,0.13,-0.03,0.00,0.03,0.97
3,-0.08,0.32,0.11,-0.39,-0.00,0.05,0.07,0.11,-0.70,-0.00,...,0.03,0.06,0.03,-0.10,-0.28,0.17,-0.08,0.06,0.73,0.21
4,0.00,0.06,0.47,-0.53,-0.34,0.24,-0.06,-0.07,0.26,-0.06,...,-0.09,0.03,-0.31,-0.37,-0.24,0.09,-0.02,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.40,0.20,-0.38,0.02,0.43,0.08,0.04,-0.09,0.06,...,0.05,-0.21,-0.09,-0.31,-0.25,0.20,0.38,0.01,0.12,0.87
24779,0.08,0.12,0.50,-0.44,-0.31,0.41,-0.06,0.27,0.14,0.02,...,-0.04,-0.45,-0.27,-0.01,-0.03,0.22,0.08,0.01,0.10,0.89
24780,-0.08,0.28,0.22,-0.41,-0.41,0.17,-0.02,0.02,0.32,-0.20,...,-0.17,-0.10,-0.39,-0.12,-0.22,0.34,0.10,0.04,0.39,0.57
24781,0.04,0.13,0.80,-0.28,-0.26,0.26,0.08,0.06,0.10,-0.46,...,-0.12,-0.01,-0.20,-0.41,-0.02,0.05,0.12,0.01,0.11,0.88


In [24]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,-0.11,0.31,0.35,-0.49,-0.07,0.55,-0.28,0.41,-0.01,0.26,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.09,-0.02,0.22,-0.41,-0.36,0.09,-0.06,0.10,0.06,-0.08,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.04,0.08,0.33,-0.47,-0.28,0.48,-0.20,-0.01,0.03,-0.19,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,-0.08,0.32,0.11,-0.39,-0.00,0.05,0.07,0.11,-0.70,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.06,0.47,-0.53,-0.34,0.24,-0.06,-0.07,0.26,-0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.40,0.20,-0.38,0.02,0.43,0.08,0.04,-0.09,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.08,0.12,0.50,-0.44,-0.31,0.41,-0.06,0.27,0.14,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.08,0.28,0.22,-0.41,-0.41,0.17,-0.02,0.02,0.32,-0.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.04,0.13,0.80,-0.28,-0.26,0.26,0.08,0.06,0.10,-0.46,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [25]:
y = dataframe['target']
X = dataframe
del X['target']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:33<00:00,  3.23s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.25
DecisionTreeClassifier,0.99,0.90,None,0.99,1.96
BaggingClassifier,0.99,0.85,None,0.99,11.64
LinearSVC,0.99,0.81,None,0.99,2.22
CalibratedClassifierCV,0.99,0.81,None,0.99,11.15
LogisticRegression,0.99,0.80,None,0.99,0.99
PassiveAggressiveClassifier,0.98,0.77,None,0.98,0.50
Perceptron,0.98,0.76,None,0.98,0.50
GaussianNB,0.83,0.76,None,0.86,0.19


In [26]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

Processing Fold 1 ...


100%|██████████| 29/29 [02:52<00:00,  5.95s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [02:51<00:00,  5.93s/it]


Processing Fold 3 ...


...

Processing Fold 9 ...


100%|██████████| 29/29 [02:47<00:00,  5.78s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [02:45<00:00,  5.71s/it]


,Fold,Model,Accuracy,F1-Score
0,1,LinearDiscriminantAnalysis,0.98,0.98
1,1,LinearSVC,0.99,0.99
2,1,DecisionTreeClassifier,0.99,0.99
3,1,BaggingClassifier,0.99,0.99
4,1,LogisticRegression,0.99,0.99
...,...,...,...,...
295,NaN,NaN,NaN,NaN
296,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN
298,NaN,NaN,NaN,NaN


In [27]:
result = result.dropna()
result

,Fold,Model,Accuracy,F1-Score
0,1,LinearDiscriminantAnalysis,0.98,0.98
1,1,LinearSVC,0.99,0.99
2,1,DecisionTreeClassifier,0.99,0.99
3,1,BaggingClassifier,0.99,0.99
4,1,LogisticRegression,0.99,0.99
...,...,...,...,...
245,10,AdaBoostClassifier,0.65,0.52
246,10,LabelSpreading,0.36,0.33
247,10,LabelPropagation,0.36,0.33
248,10,LGBMClassifier,0.26,0.16


In [28]:
result.to_csv("results/10-Fold-glove-davidson.csv")

## BERT

In [29]:
data = pd.read_csv("datasets/edited/BERT_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,101,19387,9815,19454,21818,2135,2450,17612,4550,2160,...,0,0,0,0,0,0,0,0.07,0.41,0.52
1,101,19387,19875,4402,2860,2879,23755,3147,5939,3654,...,0,0,0,0,0,0,0,0.03,0.43,0.54
2,101,19387,24471,18824,11253,23544,4830,27767,19387,24829,...,0,0,0,0,0,0,0,0.00,0.03,0.97
3,101,19387,1039,1043,5143,20022,2241,2298,2066,25283,...,0,0,0,0,0,0,0,0.06,0.73,0.21
4,101,19387,21882,7556,3217,8296,2015,4485,2963,2995,...,0,0,0,0,0,0,0,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,101,14163,8322,2546,1999,4682,2166,19895,2075,21944,...,0,0,0,0,0,0,0,0.01,0.12,0.87
24779,101,2175,3338,3308,2540,3336,3298,2417,18278,4689,...,0,0,0,0,0,0,0,0.01,0.10,0.89
24780,101,2402,10131,10587,4521,23755,9152,13871,27225,2066,...,0,0,0,0,0,0,0,0.04,0.39,0.57
24781,101,2017,2226,2288,3748,7743,2425,2378,4682,102,...,0,0,0,0,0,0,0,0.01,0.11,0.88


In [30]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,101,19387,9815,19454,21818,2135,2450,17612,4550,2160,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,101,19387,19875,4402,2860,2879,23755,3147,5939,3654,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,101,19387,24471,18824,11253,23544,4830,27767,19387,24829,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,101,19387,1039,1043,5143,20022,2241,2298,2066,25283,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,101,19387,21882,7556,3217,8296,2015,4485,2963,2995,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,101,14163,8322,2546,1999,4682,2166,19895,2075,21944,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,101,2175,3338,3308,2540,3336,3298,2417,18278,4689,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,101,2402,10131,10587,4521,23755,9152,13871,27225,2066,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,101,2017,2226,2288,3748,7743,2425,2378,4682,102,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [31]:
y = dataframe['target']
X = dataframe
del X['target']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:35<00:00,  3.30s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.99,None,0.99,0.24
BaggingClassifier,0.99,0.92,None,0.99,1.66
DecisionTreeClassifier,0.99,0.90,None,0.99,0.36
LinearSVC,0.99,0.89,None,0.99,3.37
LogisticRegression,0.99,0.88,None,0.99,0.87
CalibratedClassifierCV,0.99,0.85,None,0.99,46.08
PassiveAggressiveClassifier,0.98,0.85,None,0.98,0.45
Perceptron,0.98,0.84,None,0.98,0.41
NearestCentroid,0.86,0.83,None,0.87,0.19


In [32]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

Processing Fold 1 ...


100%|██████████| 29/29 [02:55<00:00,  6.07s/it]


Processing Fold 2 ...


100%|██████████| 29/29 [03:02<00:00,  6.28s/it]


Processing Fold 3 ...


...

Processing Fold 9 ...


100%|██████████| 29/29 [02:57<00:00,  6.11s/it]


Processing Fold 10 ...


100%|██████████| 29/29 [02:57<00:00,  6.12s/it]


,Fold,Model,Accuracy,F1-Score
0,1,LinearDiscriminantAnalysis,0.98,0.98
1,1,LogisticRegression,0.99,0.99
2,1,LinearSVC,0.99,0.99
3,1,BaggingClassifier,1.00,1.00
4,1,CalibratedClassifierCV,0.99,0.99
...,...,...,...,...
295,NaN,NaN,NaN,NaN
296,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN
298,NaN,NaN,NaN,NaN


In [33]:
result = result.dropna()
result

,Fold,Model,Accuracy,F1-Score
0,1,LinearDiscriminantAnalysis,0.98,0.98
1,1,LogisticRegression,0.99,0.99
2,1,LinearSVC,0.99,0.99
3,1,BaggingClassifier,1.00,1.00
4,1,CalibratedClassifierCV,0.99,0.99
...,...,...,...,...
245,10,RidgeClassifierCV,0.95,0.93
246,10,GaussianNB,0.44,0.42
247,10,QuadraticDiscriminantAnalysis,0.40,0.44
248,10,AdaBoostClassifier,0.65,0.52


In [34]:
result.to_csv("results/10-Fold-BERT-davidson.csv")